In [4]:
#%%
import requests
import pandas as pd
import numpy as np
# Define headers and parameters

with open('tide_api_key') as f:
    api_key = f.read()


headers = {'Ocp-Apim-Subscription-Key': api_key,}

# Base URL
base_url = 'https://admiraltyapi.azure-api.net/uktidalapi/api/V1/Stations'

try:
    # Make the GET request
    response = requests.get(base_url, headers=headers)

    # Check the response
    response.raise_for_status()
    
   # Parse the JSON response
    data = response.json()

    # Extract the necessary values
    rows = []
    for feature in data['features']:
        longitude, latitude = feature['geometry']['coordinates']
        name = feature['properties']['Name']
        continuous_heights = feature['properties']['ContinuousHeightsAvailable']
        
        rows.append([latitude, longitude, name, continuous_heights])

# Convert to a pandas DataFrame
        df = pd.DataFrame(rows, columns=['Latitude', 'Longitude', 'Name', 'ContinuousHeightsAvailable'])

    # Display the DataFrame
    df.head()
except requests.exceptions.RequestException as e:
    print(e)

In [5]:
df.head(5)

,Latitude,Longitude,Name,ContinuousHeightsAvailable
0,57.800000,-8.566666,Hirta (Bagh A' Bhaile),True
1,57.600000,-13.683333,Rockall,True
2,50.450000,-4.200000,Cargreen,True
3,50.366666,-4.183333,PLYMOUTH (DEVONPORT),True
4,50.350000,-4.450000,Looe,True


In [6]:
# Define haversine to calculate distance
def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371.0

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

def closest_station(df, lat, lon):
    # Compute distance for each station
    df['distance'] = haversine(lat, lon, df['Latitude'], df['Longitude'])

    # Filter for ContinuousHeightsAvailable and find the station with the minimum distance
    closest = df[df['ContinuousHeightsAvailable'] == True].nsmallest(1, 'distance')
    
    return closest['Name']

given_lat, given_lon = 50.220564, -4.801677
print(closest_station(df, given_lat, given_lon))

22    St. Germans
Name: Name, dtype: object
